In [2]:
import numpy as np
import open3d as o3d
import copy

from collections import defaultdict
from tqdm import tqdm

In [4]:
def load_and_repair_mesh(filepath):
    # Загружаем исходную сетку
    mesh = o3d.io.read_triangle_mesh(filepath)
    
    # Проверяем, есть ли проблема с вершинами
    vertices = np.asarray(mesh.vertices)
    triangles = np.asarray(mesh.triangles)
       
    # Создаем словарь для устранения дубликатов вершин
    vertex_map = {}
    new_vertices = []
    new_triangles = []
    
    # Хеш-функция для вершин (учет возможных погрешностей float)
    def vertex_hash(v):
        return tuple(np.round(v, 6))
    
    # Строим отображение старых вершин в новые
    for idx, v in enumerate(vertices):
        v_hash = vertex_hash(v)
        if v_hash not in vertex_map:
            vertex_map[v_hash] = len(new_vertices)
            new_vertices.append(v)
    
    # Перестраиваем треугольники с новыми индексами
    for tri in triangles:
        new_tri = [
            vertex_map[vertex_hash(vertices[tri[0]])],
            vertex_map[vertex_hash(vertices[tri[1]])],
            vertex_map[vertex_hash(vertices[tri[2]])]
        ]
        new_triangles.append(new_tri)
    
    # Создаем новую сетку
    repaired_mesh = o3d.geometry.TriangleMesh()
    repaired_mesh.vertices = o3d.utility.Vector3dVector(np.array(new_vertices))
    repaired_mesh.triangles = o3d.utility.Vector3iVector(np.array(new_triangles))
    
    # Вычисляем нормали для корректного отображения
    repaired_mesh.compute_vertex_normals()
        
    return repaired_mesh

In [5]:
mesh_a = load_and_repair_mesh("model.stl")
mesh_b = load_and_repair_mesh("sphere.stl")

if not mesh_a.has_vertex_normals():
    mesh_a.compute_vertex_normals()
if not mesh_b.has_vertex_normals():
    mesh_b.compute_vertex_normals()

In [6]:
def triangle_area(v1, v2, v3):
    """Вычисление площади треугольника"""
    return 0.5 * np.linalg.norm(np.cross(v2 - v1, v3 - v1))

def center_of_mass(mesh):
    """Вычисление центра масс меша"""
    triangles = np.asarray(mesh.triangles)
    vertices = np.asarray(mesh.vertices)
    
    # Получаем вершины каждого треугольника
    tri_vertices = vertices[triangles]
    
    # Центры треугольников
    centers = np.mean(tri_vertices, axis=1)
    
    # Площади треугольников
    areas = np.array([triangle_area(t[0], t[1], t[2]) for t in tri_vertices])
    
    # Центр масс
    return np.sum(centers * areas.reshape(-1, 1), axis=0) / np.sum(areas)

In [ ]:
ITERS = 10

def normalize(vector):
    if not np.all(vector):
        raise ZeroDivisionError

    return vector / np.linalg.norm(vector)


def get_adjacent_vertices(mesh):
    """Построение словаря смежных вершин"""
    triangles = np.asarray(mesh.triangles)
    adjacency_dict = defaultdict(set)

    for tri in triangles:
        v0, v1, v2 = tri

        # Добавляем связи между вершинами треугольника
        adjacency_dict[v0].update([v1, v2])
        adjacency_dict[v1].update([v0, v2])
        adjacency_dict[v2].update([v0, v1])

    return adjacency_dict


def relax_mesh(mesh, iter_limit=ITERS):
    """Релаксация меша с сохранением топологии"""
    # Получаем копию исходного меша
    relaxed_mesh = copy.deepcopy(mesh)
    
    # Получаем вершины и треугольники
    vertices = np.asarray(relaxed_mesh.vertices)
    adj = get_adjacent_vertices(mesh)
    
    # Релаксация
    for _ in range(iter_limit):
        prev_vertices = copy.deepcopy(vertices)

        for i in range(len(vertices)):
            neighbors = list(adj[i])
            try:
                vertices[i] = normalize(np.mean(prev_vertices[neighbors], axis=0))
            except:
                pass
    
    # Обновляем нормали
    relaxed_mesh.compute_vertex_normals()
    return relaxed_mesh


def parametrize_mesh(mesh):
    """Параметризация меша на сфере"""
    # Создаем копию меша
    mesh_copy = copy.deepcopy(mesh)

    # Вычисляем центр масс
    center = center_of_mass(mesh_copy)

    # Переносим вершины в начало координат и нормализуем
    vertices = np.asarray(mesh_copy.vertices)
    vertices -= center
    norms = np.linalg.norm(vertices, axis=1)
    norms[norms == 0] = 1  # Избегаем деления на ноль #TODO: troubleshoot
    vertices = vertices / norms.reshape(-1, 1)

    # Обновляем вершины и нормали
    mesh_copy.vertices = o3d.utility.Vector3dVector(vertices)
    mesh_copy.compute_vertex_normals()

    # Применяем релаксацию
    return relax_mesh(mesh_copy, ITERS)

In [9]:
def overlap_meshes(source_mesh, target_mesh):
    source_pcd = o3d.geometry.PointCloud()
    source_pcd.points = source_mesh.vertices
    target_pcd = o3d.geometry.PointCloud()
    target_pcd.points = target_mesh.vertices
    
    source_pcd.estimate_normals()
    target_pcd.estimate_normals()

    trans_init = np.identity(4)
    threshold = 0.02 * 2
    reg_p2l = o3d.pipelines.registration.registration_icp(
        source_pcd, target_pcd, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPlane(),
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=200)
    )
    source_pcd.transform(reg_p2l.transformation)
    source_mesh.vertices = source_pcd.points

In [10]:
def add_wireframe_to_geo(mesh, geo, color = [0, 0, 0]):
    wireframe = o3d.geometry.LineSet.create_from_triangle_mesh(mesh)
    pcd = o3d.geometry.PointCloud()
    pcd.points = mesh.vertices

    pcd.paint_uniform_color(color)
    wireframe.paint_uniform_color(color)

    geo.append(wireframe)
    geo.append(pcd)

In [110]:
def arcs(mesh):
    vertices = np.asarray(mesh.vertices)
    for tri in np.asarray(mesh.triangles):
        yield [vertices[tri[0]], vertices[tri[1]]]
        yield [vertices[tri[1]], vertices[tri[2]]]
        yield [vertices[tri[2]], vertices[tri[0]]]

def find_parametrized_mesh_intersections(mesh_a: o3d.geometry.TriangleMesh, mesh_b: o3d.geometry.TriangleMesh):
    result = np.ndarray((0, 3), dtype=float)
    
    for arc_a in tqdm(arcs(mesh_a), total=len(mesh_a.triangles) * 3):
        normal_a = np.cross(arc_a[0], arc_a[1])
        for arc_b in arcs(mesh_b):
            if np.dot(arc_a[0], arc_b[0]) < 0:
                continue

            normal_b = np.cross(arc_b[0], arc_b[1])
            d = np.cross(normal_a, normal_b)

            if np.isclose(np.linalg.norm(d), 0):
                continue
            
            if np.dot(d, arc_a[0]) < 0:
                d = -d

            if np.dot(np.cross(arc_a[0], d), np.cross(arc_a[1], d)) >= 0:
                continue
            
            if np.dot(np.cross(arc_b[0], d), np.cross(arc_b[1], d)) >= 0:
                continue
        
            result = np.append(result, [normalize(d)], axis=0)
    
    return result

In [111]:
parametrized_mesh_a = parametrize_mesh(mesh_a)
parametrized_mesh_b = parametrize_mesh(mesh_b)
overlap_meshes(parametrized_mesh_b, parametrized_mesh_a)

In [113]:
inter_points = find_parametrized_mesh_intersections(parametrized_mesh_a, parametrized_mesh_b)

  0%|          | 0/768 [00:00<?, ?it/s]

100%|██████████| 768/768 [02:29<00:00,  5.14it/s]


In [114]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(inter_points)
pcd.paint_uniform_color([0, 1, 0])

PointCloud with 5988 points.

In [ ]:
geo = [pcd]

add_wireframe_to_geo(parametrized_mesh_a, geo, [0, 0, 1])
add_wireframe_to_geo(parametrized_mesh_b, geo, [1, 0, 0])
o3d.visualization.draw_geometries(geo, mesh_show_wireframe=True)

2025-05-29 19:55:37.152 python[66757:1158396] +[IMKClient subclass]: chose IMKClient_Modern
2025-05-29 19:55:37.152 python[66757:1158396] +[IMKInputSession subclass]: chose IMKInputSession_Modern
